# VicunaAGI Project
- [Oobabooga Text Generation Web UI](https://github.com/oobabooga/text-generation-webui)
- [Vicuna 13B Model](https://github.com/lm-sys/FastChat/#vicuna-weights)
- Modified [BabyAGI](https://github.com/yoheinakajima/babyagi) called [Agent-LLM](https://github.com/Josh-XT/Agent-LLM)

# Setup Instructions

This notebook is intended for use in Ubuntu Linux and assumes you have a GPU capable of running this model with the appropriate drivers installed.

## Install Initial Requirements

In [ ]:
git clone https://github.com/Josh-XT/Agent-LLM
pip install -r "Agent-LLM/requirements.txt"
pip install -r "requirements.txt"

## Install Conda

In [ ]:
curl -sL "https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh" > "Miniconda3.sh"
sudo bash Miniconda3.sh
sudo apt install build-essential
conda create -n textgen python=3.10.9

## Download Oobabooga Text Generation Web UI

In [ ]:
git clone https://github.com/oobabooga/text-generation-webui
cd text-generation-webui
pip install -r requirements.txt

## Download Vicuna 13B Model

In [ ]:
cd ../text-generation-webui/
python3 download-model.py FIND-A/VICUNA-MODEL-ON-HUGGINGFACE

## Run Oobabooga Text Generation Web UI

In [ ]:
conda activate textgen
python3 server.py --model eachadea_vicuna-13b-1.1 --listen --no-stream

Access at http://localhost:7860/?__theme=dark once running.

## Set The AI Objectives
Set your ``OBJECTIVE`` then run `Auto-LLM`

In [ ]:
cd Agent-LLM
python3 main.py "Solve world hunger"